In [131]:
# Allow multiple outputs in a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Analysis of SleepCycle data

Let's start by downloading the data from SleepSecure(TM). This might take a while the first time, as the data is downloaded from a *relatively* slow server:

In [ ]:
from quelf.sleepcycle import SleepCycle

sc = SleepCycle()
data = sc.data

How large is our dataset?

In [ ]:
len(data)

What does our data consist of?

In [ ]:
data.columns

What does our latest data point look like?

In [ ]:
data.loc[0]

## Data processing
Let's calculate how long each sleep recording lasts:

In [ ]:
time_slept = data['stop'] - data['start']
data['time_slept'] = time_slept
time_slept.describe()

The dataset might contain some outliers. If these maximum and/or minimum values look incorrect to you, you might want to delete the datapoints from the SleepCycle app, delete the contents of the ``data`` directory, and rerun the notebook.

In [ ]:
# Show the maximum and minimum time slept, and on which date they occur
data[data['time_slept'] == data['time_slept'].max()]
data[data['time_slept'] == data['time_slept'].min()]

In [ ]:
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Fill the width of a 1920x1080 screen when plotting the entire dataset
plt.rcParams['figure.figsize'] = [25, 6]

# Use FiveThirtyEight matplotlib theme
plt.style.use('fivethirtyeight')

# Calculate time went to bed, as a offset relative to 18:00
went_to_bed = (data['start'].dt.hour + (data['start'].dt.second / 60) - 18) % 24

# Decimal hours slept
hours_slept = time_slept.dt.total_seconds() / (60 * 60)

# Create colormap relative to hours slept
norm = (hours_slept - hours_slept.min()) / hours_slept.max()
cmap = cm.get_cmap()
color_map = cmap(norm)

# Bottom of barplot relative to 18:00, height relative to hours slept
ax = plt.bar(data['start'].values, hours_slept, bottom=went_to_bed, width=1.125, color=color_map)

plt.xlabel('Date')
plt.ylabel('Hours slept')

# Map ticks relative to 18:00
plt.yticks((np.arange(2, 25, 2) - 18) % 24, (np.arange(2, 25, 2)))

plt.show()